### Utils

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils import data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
class Dataset(data.Dataset):
    def __init__(self, input_length, mid, output_length, direc, task_list, sample_list, stack = False):
        self.input_length = input_length
        self.mid = mid
        self.output_length = output_length
        self.direc = direc
        self.task_list = task_list
        self.sample_list = sample_list
        self.stack = stack
        try:
            self.data_lists = [torch.load(self.direc + "/raw_data_" + str(idx[0]) + "_" + str(idx[1]) + ".pt") for idx in task_list]
        except:
            self.data_lists = [torch.load(self.direc + "/raw_data_" + str(idx) + ".pt") for idx in task_list]

    def __len__(self):
        return len(self.task_list) * len(self.sample_list)

    def __getitem__(self, index):
        task_idx = index // len(self.sample_list)
        sample_idx = index % len(self.sample_list)        
        y = self.data_lists[task_idx][(self.sample_list[sample_idx]+self.mid):(self.sample_list[sample_idx]+self.mid+self.output_length)] 
        if not self.stack:
            x = self.data_lists[task_idx][(self.mid-self.input_length+self.sample_list[sample_idx]):(self.mid+self.sample_list[sample_idx])]
        else:
            x = self.data_lists[task_idx][(self.mid-self.input_length+self.sample_list[sample_idx]):(self.mid+self.sample_list[sample_idx])].reshape(-1, y.shape[-2], y.shape[-1])     
        return x.float(), y.float()
    
def train_epoch(train_loader, model, optimizer, loss_function):
    train_mse = []
    for xx, yy in train_loader:
        xx = xx.to(device)
        yy = yy.to(device)
        loss = 0
        for y in yy.transpose(0,1):
            im = model(xx)
            xx = torch.cat([xx[:, im.shape[1]:], im], 1)
            loss += loss_function(im, y)  
        train_mse.append(loss.item()/yy.shape[1]) 
        try:
            weight_constraint = loss_function(model.module.get_weight_constraint(), torch.tensor(0).float().cuda())
            loss += weight_constraint
        except:
            pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_rmse = round(np.sqrt(np.mean(train_mse)),5)
    return train_rmse

def eval_epoch(valid_loader, model, loss_function):
    valid_mse = []
    preds = []
    trues = []
    with torch.no_grad():
        for xx, yy in valid_loader:
            xx = xx.to(device)
            yy = yy.to(device)
            loss = 0
            ims = []
            for y in yy.transpose(0,1):
                im = model(xx)
                xx = torch.cat([xx[:, im.shape[1]:], im], 1)
                loss += loss_function(im, y)
                ims.append(im.cpu().data.numpy())
            ims = np.array(ims).transpose(1,0,2,3,4)
            preds.append(ims)
            trues.append(yy.cpu().data.numpy())
            valid_mse.append(loss.item()/yy.shape[1])
        preds = np.concatenate(preds, axis = 0)  
        trues = np.concatenate(trues, axis = 0)  
        valid_rmse = round(np.sqrt(np.mean(valid_mse)), 5)
    return valid_rmse, preds, trues

### Extra (needed)

due to conversion to ecsnn the compute_basis_param function changed

In [ ]:

def get_grid_coords(kernel_size: int, dilation: int = 1):
    
    actual_size = dilation * (kernel_size -1) + 1
    
    origin = actual_size / 2 - 0.5
    
    points = []
    
    for y in range(kernel_size):
        y *= dilation
        for x in range(kernel_size):
            x *= dilation
            p = (x - origin, -y + origin)
            points.append(p)
    
    points = np.array(points)
    assert points.shape == (kernel_size ** 2, 2), points.shape
    return points.T

### Models

In [ ]:
import torch
import escnn
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from escnn.nn.modules.conv.r2convolution import compute_basis_params     #This is the escnn version
from escnn.nn.modules.basismanager.basisexpansion_singleblock import block_basisexpansion  #This is the escnn version
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



########### Regular CNN ########
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ConvBlock, self).__init__()
        self.conv =nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size-1)//2),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ) 
        
    def forward(self, x):
        return self.conv(x)
    
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_dim, kernel_size, num_layers):
        super(ConvNet, self).__init__()
        self.layers = [ConvBlock(in_channels, hidden_dim, kernel_size)]
        self.layers += [ConvBlock(hidden_dim, hidden_dim, kernel_size) for i in range(num_layers-2)]
        self.layers += [nn.Conv2d(hidden_dim, out_channels, kernel_size, padding=(kernel_size-1)//2)]
        self.model = nn.Sequential(*self.layers)
        
    def forward(self, x):
        return self.model(x)
    
    

######################################################  
############ Rotationally Equivariant CNN ############
######################################################


class E2Conv(torch.nn.Module):
    def __init__(self, in_frames, out_frames, kernel_size, N):
        super(E2Conv, self).__init__()
        
        r2_act = escnn.gspaces.rot2dOnR2(N = N)
        feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.regular_repr])
        feat_type_hid = escnn.nn.FieldType(r2_act, out_frames*[r2_act.regular_repr])
        
        self.layer = escnn.nn.SequentialModule(
            escnn.nn.R2Conv(feat_type_in, feat_type_hid, kernel_size = kernel_size, padding = (kernel_size - 1)//2),
            escnn.nn.InnerBatchNorm(feat_type_hid),
            escnn.nn.ReLU(feat_type_hid)
        ) 
        
    def forward(self, xx):
        return self.layer(xx)

class E2CNN(torch.nn.Module):
    def __init__(self, in_frames, out_frames, hidden_dim, kernel_size, num_layers, N):
        super(E2CNN, self).__init__()
        r2_act = escnn.gspaces.rot2dOnR2(N = N)
        
        self.feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.irrep(1)])
        self.feat_type_hid = escnn.nn.FieldType(r2_act, hidden_dim*[r2_act.regular_repr])
        self.feat_type_out = escnn.nn.FieldType(r2_act, out_frames*[r2_act.irrep(1)])
        
        input_layer = escnn.nn.SequentialModule(
            escnn.nn.R2Conv(self.feat_type_in, self.feat_type_hid, kernel_size = kernel_size, padding = (kernel_size - 1)//2),
            escnn.nn.InnerBatchNorm(self.feat_type_hid),
            escnn.nn.ReLU(self.feat_type_hid)
        ) 
        
        layers = [input_layer]
        layers += [E2Conv(hidden_dim, hidden_dim, kernel_size, N) for i in range(num_layers-2)]
        layers += [escnn.nn.R2Conv(self.feat_type_hid, self.feat_type_out, kernel_size = kernel_size, padding = (kernel_size - 1)//2)]
        self.model = torch.nn.Sequential(*layers)
    
        
    def forward(self, xx):
        xx = escnn.nn.GeometricTensor(xx, self.feat_type_in)
        out = self.model(xx)
        return out.tensor
    
    

############ ConvNet + E2CNN ############

class ConvE2CNN(torch.nn.Module):
    def __init__(self, in_frames, out_frames, kernel_size, conv_hidden_dim, escnn_hidden_dim, conv_num_layers, escnn_num_layers,  N):
        super(ConvE2CNN, self).__init__()
        
        self.convnet = [ConvBlock(in_frames*2, conv_hidden_dim, kernel_size)]
        self.convnet += [ConvBlock(conv_hidden_dim, conv_hidden_dim, kernel_size) for i in range(conv_num_layers-2)]
        self.convnet += [ConvBlock(conv_hidden_dim, escnn_hidden_dim*N, kernel_size)]
        self.convnet = nn.Sequential(*self.convnet)
        
        r2_act = escnn.gspaces.rot2dOnR2(N = N)
        self.feat_type_hid = escnn.nn.FieldType(r2_act, escnn_hidden_dim*[r2_act.regular_repr])
        self.feat_type_out = escnn.nn.FieldType(r2_act, out_frames*[r2_act.irrep(1)])
        
        self.escnn = [E2Conv(escnn_hidden_dim, escnn_hidden_dim, kernel_size, N) for i in range(escnn_num_layers-2)]
        self.escnn += [escnn.nn.R2Conv(self.feat_type_hid, self.feat_type_out, kernel_size = kernel_size, padding = (kernel_size - 1)//2)]
        self.escnn = torch.nn.Sequential(*self.escnn)
    
        
    def forward(self, xx):
        out = self.convnet(xx)
        out = escnn.nn.GeometricTensor(out, self.feat_type_hid)
        out = self.escnn(out)
        return out.tensor
    

    

    

  

############ Relaxed Group Convolution ############
class Relaxed_LiftingConvolution(torch.nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 group_order,
                 num_filter_banks,
                 activation = True
                 ):
        super(Relaxed_LiftingConvolution, self).__init__()

        self.num_filter_banks = num_filter_banks
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.group_order = group_order
        self.activation = activation

        self.combination_weights = torch.nn.Parameter(torch.ones(num_filter_banks, group_order).float()/num_filter_banks)

        # Initialize an unconstrained kernel.
        self.weight = torch.nn.Parameter(torch.zeros(self.num_filter_banks, # Additional dimension
                                                     self.out_channels,
                                                     self.in_channels,
                                                     self.kernel_size,
                                                     self.kernel_size))
        stdv = np.sqrt(1/(self.in_channels*self.kernel_size*self.kernel_size))
        self.weight.data.uniform_(-stdv, stdv)

        # If combination_weights are equal values, then the model is still equivariant
        # self.combination_weights.data.uniform_(-stdv, stdv)
        
    def generate_filter_bank(self):
        """ Obtain a stack of rotated filters"""
        weights = self.weight.reshape(self.num_filter_banks*self.out_channels,
                                      self.in_channels,
                                      self.kernel_size,
                                      self.kernel_size)
        filter_bank = torch.stack([rot_img(weights, -np.pi*2/self.group_order*i)
                                   for i in range(self.group_order)])
        filter_bank = filter_bank.transpose(0,1).reshape(self.num_filter_banks, # Additional dimension
                                                         self.out_channels,
                                                         self.group_order,
                                                         self.in_channels,
                                                         self.kernel_size,
                                                         self.kernel_size)
        return filter_bank


    def forward(self, x):
        # input shape: [bz, #in, h, w]
        # output shape: [bz, #out, group order, h, w]

        # generate filter bank given input group order
        filter_bank = self.generate_filter_bank()

        # for each rotation, we have a linear combination of multiple filters with different coefficients.
        relaxed_conv_weights = torch.einsum("na, noa... -> oa...", self.combination_weights, filter_bank)

        # concatenate the first two dims before convolution.
        # ==============================
        x = F.conv2d(
            input=x,
            weight=relaxed_conv_weights.reshape(
                self.out_channels * self.group_order,
                self.in_channels,
                self.kernel_size,
                self.kernel_size
            ),
            padding = (self.kernel_size-1)//2
        )
        # ==============================

        # reshape output signal to shape [bz, #out, group order, h, w].
        # ==============================
        x = x.view(
            x.shape[0],
            self.out_channels,
            self.group_order,
            x.shape[-1],
            x.shape[-2]
        )
        # ==============================

        if self.activation:
            return F.relu(x)
        return x

        
class Relaxed_GroupConv(torch.nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 group_order,
                 num_filter_banks,
                 activation = True
                ):

        super(Relaxed_GroupConv, self).__init__()

        self.num_filter_banks = num_filter_banks
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.group_order = group_order
        self.activation = activation


        ## Initialize weights
        self.combination_weights = torch.nn.Parameter(torch.ones(group_order, num_filter_banks).float()/num_filter_banks/group_order)
        self.weight = torch.nn.Parameter(torch.randn(self.num_filter_banks, ##additional dimension
                                                       self.out_channels,
                                                       self.in_channels,
                                                       self.group_order,
                                                       self.kernel_size,
                                                       self.kernel_size))

        stdv = np.sqrt(1/(self.in_channels*self.kernel_size*self.kernel_size))
        self.weight.data.uniform_(-stdv, stdv)

        # If combination_weights are equal values, then the model is still equivariant
        # self.combination_weights.data.uniform_(-stdv, stdv)


    def generate_filter_bank(self):
        """ Obtain a stack of rotated and cyclic shifted filters"""
        filter_bank = []
        weights = self.weight.reshape(self.num_filter_banks*self.out_channels*self.in_channels,
                                      self.group_order,
                                      self.kernel_size,
                                      self.kernel_size)

        for i in range(self.group_order):
            # planar rotation
            rotated_filter = rot_img(weights, -np.pi*2/self.group_order*i)

            # cyclic shift
            shifted_indices = torch.roll(torch.arange(0, self.group_order, 1), shifts = i)
            shifted_rotated_filter = rotated_filter[:,shifted_indices]
            
            
            filter_bank.append(shifted_rotated_filter.reshape(self.num_filter_banks,
                                                    self.out_channels,
                                                    self.in_channels,
                                                    self.group_order,
                                                    self.kernel_size,
                                                    self.kernel_size))
        # stack
        filter_bank = torch.stack(filter_bank).permute(1,2,0,3,4,5,6)
        return filter_bank

    def forward(self, x):

        filter_bank = self.generate_filter_bank()

        relaxed_conv_weights = torch.einsum("na, aon... -> on...", self.combination_weights, filter_bank)

        x = torch.nn.functional.conv2d(
            input=x.reshape(
                x.shape[0],
                x.shape[1] * x.shape[2],
                x.shape[3],
                x.shape[4]
                ),
            weight=relaxed_conv_weights.reshape(
                self.out_channels * self.group_order,
                self.in_channels * self.group_order,
                self.kernel_size,
                self.kernel_size
            ),
            padding = (self.kernel_size-1)//2
        )

                # Reshape signal back [bz, #out * g_order, h, w] -> [bz, out, g_order, h, w]
        x = x.view(x.shape[0], self.out_channels, self.group_order, x.shape[-2], x.shape[-1])
        # ========================

        return x

class Relaxed_LiftingConvolution(torch.nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 group_order,
                 num_filter_banks,
                 activation = True
                 ):
        super(Relaxed_LiftingConvolution, self).__init__()

        self.num_filter_banks = num_filter_banks
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.group_order = group_order
        self.activation = activation

        self.combination_weights = torch.nn.Parameter(torch.ones(num_filter_banks, group_order).float()/num_filter_banks)

        # Initialize an unconstrained kernel.
        self.weight = torch.nn.Parameter(torch.zeros(self.num_filter_banks, # Additional dimension
                                                     self.out_channels,
                                                     self.in_channels,
                                                     self.kernel_size,
                                                     self.kernel_size))
        stdv = np.sqrt(1/(self.in_channels*self.kernel_size*self.kernel_size))
        self.weight.data.uniform_(-stdv, stdv)

        # If combination_weights are equal values, then the model is still equivariant
        # self.combination_weights.data.uniform_(-stdv, stdv)
        
    def generate_filter_bank(self):
        """ Obtain a stack of rotated filters"""
        weights = self.weight.reshape(self.num_filter_banks*self.out_channels,
                                      self.in_channels,
                                      self.kernel_size,
                                      self.kernel_size)
        filter_bank = torch.stack([rot_img(weights, -np.pi*2/self.group_order*i)
                                   for i in range(self.group_order)])
        filter_bank = filter_bank.transpose(0,1).reshape(self.num_filter_banks, # Additional dimension
                                                         self.out_channels,
                                                         self.group_order,
                                                         self.in_channels,
                                                         self.kernel_size,
                                                         self.kernel_size)
        return filter_bank


    def forward(self, x):
        # input shape: [bz, #in, h, w]
        # output shape: [bz, #out, group order, h, w]

        # generate filter bank given input group order
        filter_bank = self.generate_filter_bank()

        # for each rotation, we have a linear combination of multiple filters with different coefficients.
        relaxed_conv_weights = torch.einsum("na, noa... -> oa...", self.combination_weights, filter_bank)

        # concatenate the first two dims before convolution.
        # ==============================
        x = F.conv2d(
            input=x,
            weight=relaxed_conv_weights.reshape(
                self.out_channels * self.group_order,
                self.in_channels,
                self.kernel_size,
                self.kernel_size
            ),
            padding = (self.kernel_size-1)//2
        )
        # ==============================

        # reshape output signal to shape [bz, #out, group order, h, w].
        # ==============================
        x = x.view(
            x.shape[0],
            self.out_channels,
            self.group_order,
            x.shape[-1],
            x.shape[-2]
        )
        # ==============================

        if self.activation:
            return F.relu(x)
        return x
    
class RelaxedGroupEquivariantCNN(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, hidden_dim, group_order, num_gconvs, num_filter_banks, vel_inp = True):
        super().__init__()

        # First transform \rho_1 to regular representations. 
        theta = torch.tensor(2*np.pi/group_order).float()
        self.lift_coefs = torch.tensor([[torch.cos(theta*i), torch.sin(theta*i)] for i in range(group_order)]).float().to(device)
        
        if vel_inp:
            self.gconvs = [Relaxed_GroupConv(in_channels = in_channels,
                                            out_channels = hidden_dim,
                                            kernel_size = kernel_size,
                                            group_order = group_order,
                                            num_filter_banks = num_filter_banks,
                                            activation = True)]
        else:
            self.gconvs = [Relaxed_LiftingConvolution(in_channels = in_channels,
                                                      out_channels = hidden_dim,
                                                      kernel_size = kernel_size,
                                                      group_order = group_order,
                                                      num_filter_banks = num_filter_banks,
                                                      activation = True)]

        for i in range(num_gconvs-2):
            self.gconvs.append(Relaxed_GroupConv(in_channels = hidden_dim,
                                                out_channels = hidden_dim,
                                                kernel_size = kernel_size,
                                                group_order = group_order,
                                                num_filter_banks = num_filter_banks,
                                                activation = True))
            
        self.gconvs.append(Relaxed_GroupConv(in_channels = hidden_dim,
                                            out_channels = out_channels,
                                            kernel_size = kernel_size,
                                            group_order = group_order,
                                            num_filter_banks = num_filter_banks,
                                            activation = False))

        self.gconvs = torch.nn.Sequential(*self.gconvs)


        self.vel_inp = vel_inp
        self.group_order = group_order

    def forward(self, x, target_length = 1):
        if self.vel_inp and len(x.shape) == 4:
            x = x.reshape(x.shape[0], x.shape[1]//2, 2, x.shape[2], x.shape[3])
        preds = []
        for i in range(target_length):
            if self.vel_inp:
                x = torch.einsum("bivhw, nv->binhw", x, self.lift_coefs)
            out = self.gconvs(x)
            if self.vel_inp:
                out = torch.einsum("binhw, nv->bivhw", out, self.lift_coefs)
            else:
                out = out.mean(2)
            x = torch.cat([x[:, 1:], out], 1)
            preds.append(out)
            
        outs = torch.cat(preds, dim=1)
        outs = outs.reshape(outs.shape[0], -1, outs.shape[-2], outs.shape[-1])
        return outs

    
def rot_img(x, theta):
    """ Rotate 2D images
    Args:
        x : input images with shape [N, C, H, W]
        theta: angle
    Returns:
        rotated images
    """
    # Rotation Matrix (2 x 3)
    rot_mat = torch.FloatTensor([[np.cos(theta), -np.sin(theta), 0],
                                 [np.sin(theta), np.cos(theta), 0]]).to(x.device)

    # The affine transformation matrices should have the shape of N x 2 x 3
    rot_mat = rot_mat.repeat(x.shape[0],1,1)

    # Obtain transformed grid
    # grid is the coordinates of pixels for rotated image
    # F.affine_grid assumes the origin is in the middle
    # and it rotates the positions of the coordinates
    # r(f(x)) = f(r^-1 x)
    grid = F.affine_grid(rot_mat, x.size(), align_corners=False).float().to(x.device)
    x = F.grid_sample(x, grid)
    return x

def rot_vector(x, theta):
    #x has the shape [c x 2 x h x w]
    rho = torch.FloatTensor([[np.cos(theta), -np.sin(theta)],
                             [np.sin(theta), np.cos(theta)]])
    out = torch.einsum("ab, bc... -> ac...",(rho, x.transpose(0,1))).transpose(0,1)
    return out
    


 
############ Relaxed Steerable Convolution ############

class Relaxed_Rot_SteerConv(torch.nn.Module):
    def __init__(self, in_frames, out_frames, kernel_size, N, first_layer = False, last_layer = False):
        super(Relaxed_Rot_SteerConv, self).__init__()
        r2_act = escnn.gspaces.rot2dOnR2(N = N)
        self.last_layer = last_layer
        self.first_layer = first_layer
        self.kernel_size = kernel_size
        
        if self.first_layer:
            self.feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.irrep(1)])
        else:
            self.feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.regular_repr])
            
        if self.last_layer:
            self.feat_type_hid = escnn.nn.FieldType(r2_act, out_frames*[r2_act.irrep(1)])
        else:
            self.feat_type_hid = escnn.nn.FieldType(r2_act, out_frames*[r2_act.regular_repr])
            
        if not last_layer:
            self.norm = escnn.nn.InnerBatchNorm(self.feat_type_hid)
            self.relu = escnn.nn.ReLU(self.feat_type_hid)
        
        
        basis_filter, rings, sigma = compute_basis_params(kernel_size = kernel_size)
        grid = get_grid_coords(kernel_size=kernel_size)
        i_repr = self.feat_type_in._unique_representations.pop()
        o_repr = self.feat_type_hid._unique_representations.pop()
        basis = self.feat_type_in.gspace.build_kernel_basis(i_repr, o_repr, sigma, rings, maximum_frequency = 5)
        block_expansion = block_basisexpansion(basis, grid, basis_filter, recompute=False)

        
        self.basis_kernels = block_expansion.sampled_basis.to(device)
        
        
        stdv = np.sqrt(1/(in_frames*kernel_size*kernel_size))
        self.relaxed_weights = nn.Parameter(torch.ones(out_frames, self.basis_kernels.shape[0], in_frames, kernel_size**2).float().to(device))
        self.relaxed_weights.data.uniform_(-stdv, stdv)

        self.bias = nn.Parameter(torch.zeros(out_frames*self.basis_kernels.shape[1]).to(device))
        self.bias.data.uniform_(-stdv, stdv)
        
        # self.relaxed_weights = nn.Parameter(torch.ones(out_frames, self.basis_kernels.shape[0], in_frames, kernel_size**2).float().to(device)/self.basis_kernels.shape[0]/(kernel_size**2))
        # self.bias = nn.Parameter(torch.zeros(out_frames*self.basis_kernels.shape[1]).to(device))

        
    def get_weight_constraint(self):
        return torch.mean(torch.abs(self.relaxed_weights[...,:-1] - self.relaxed_weights[...,1:])) #torch.roll()

    def forward(self, x):
        conv_filters = torch.einsum('bpqk,obik->opiqk', self.basis_kernels.to(self.relaxed_weights.device), self.relaxed_weights) 
        conv_filters = conv_filters.reshape(conv_filters.shape[0]*conv_filters.shape[1],
                                            conv_filters.shape[2]*conv_filters.shape[3], 
                                            self.kernel_size, self.kernel_size)
        
        if not self.last_layer:
            out = F.conv2d(x, conv_filters, self.bias, padding = 1)
            return self.relu(escnn.nn.GeometricTensor(out, self.feat_type_hid)).tensor#self.norm(
        else:
            return F.conv2d(x, conv_filters, self.bias, padding = 1)
        
class Relaxed_Rot_SteerConvNet(torch.nn.Module):
    def __init__(self, in_frames, out_frames, hidden_dim, kernel_size, num_layers, N, alpha = 1):
        super(Relaxed_Rot_SteerConvNet, self).__init__()
        self.alpha = alpha

        layers = [Relaxed_Rot_SteerConv(in_frames = in_frames, out_frames = hidden_dim, 
                                 kernel_size = kernel_size, N = N, 
                                 first_layer = True, last_layer = False)]
        
        layers += [Relaxed_Rot_SteerConv(in_frames = hidden_dim, out_frames = hidden_dim, 
                                  kernel_size = kernel_size, N = N, 
                                  first_layer = False, last_layer = False) 
                   for i in range(num_layers-2)]
        
        layers += [Relaxed_Rot_SteerConv(in_frames = hidden_dim, out_frames = out_frames, 
                                  kernel_size = kernel_size, N = N, 
                                  first_layer = False, last_layer = True) ]
        self.model = torch.nn.Sequential(*layers)
        
    def rot_vector(self, inp, theta):
        #inp shape: c x 2 x 64 x 64
        theta = torch.tensor(theta).float().to(device)
        rot_matrix = torch.tensor([[torch.cos(theta), -torch.sin(theta)], [torch.sin(theta), torch.cos(theta)]]).float().to(device)
        out = torch.einsum("ab, bc... -> ac...",(rot_matrix, inp.transpose(0,1))).transpose(0,1)
        return out
    
    def get_rot_mat(self, theta):
        theta = torch.tensor(theta).float().to(device)
        return torch.tensor([[torch.cos(theta), -torch.sin(theta), 0],
                             [torch.sin(theta), torch.cos(theta), 0]]).float().to(device)

    def rot_img(self, x, theta):
        rot_mat = self.get_rot_mat(theta)[None, ...].float().repeat(x.shape[0],1,1)
        grid = F.affine_grid(rot_mat, x.size()).float()
        x = F.grid_sample(x, grid)
        return x.float()
   
    def get_weight_constraint(self):
        return self.alpha * sum([layer.get_weight_constraint() for layer in self.model])
    
        
    def forward(self, xx):
        return self.model(xx)

    
    
############ Relaxed Rotation and Translation Steerable Convolution ############

class Relaxed_R_SteerConv(torch.nn.Module):
    def __init__(self, in_frames, out_frames, kernel_size, N, first_layer = False, last_layer = False):
        super(Relaxed_R_SteerConv, self).__init__()
        r2_act = escnn.gspaces.rot2dOnR2(N = N)
        self.last_layer = last_layer
        self.first_layer = first_layer
        self.kernel_size = kernel_size
        
        if self.first_layer:
            self.feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.irrep(1)]) 
        else:
            self.feat_type_in = escnn.nn.FieldType(r2_act, in_frames*[r2_act.regular_repr])
            
        if self.last_layer:
            self.feat_type_hid = escnn.nn.FieldType(r2_act, out_frames*[r2_act.irrep(1)])
        else:
            self.feat_type_hid = escnn.nn.FieldType(r2_act, out_frames*[r2_act.regular_repr]) 
            
        if not last_layer:
            self.norm = escnn.nn.InnerBatchNorm(self.feat_type_hid)
            self.relu = escnn.nn.ReLU(self.feat_type_hid)
        
        
        basis_filter, rings, sigma, maximum_frequency = compute_basis_params(kernel_size = kernel_size)
        grid = get_grid_coords(kernel_size=kernel_size)
        i_repr = self.feat_type_in._unique_representations.pop()
        o_repr = self.feat_type_hid._unique_representations.pop()
        basis = self.feat_type_in.gspace.build_kernel_basis(i_repr, o_repr, sigma, rings, maximum_frequency = 5)
        block_expansion = block_basisexpansion(basis, grid, basis_filter, recompute=False)

        
        self.basis_kernels = block_expansion.sampled_basis.to(device)  
        
        
        stdv = np.sqrt(1/(in_frames*kernel_size*kernel_size))
        self.relaxed_weights = nn.Parameter(torch.ones(out_frames, self.basis_kernels.shape[0], in_frames, kernel_size**2).float().to(device))
        self.relaxed_weights.data.uniform_(-stdv, stdv)

        self.bias = nn.Parameter(torch.zeros(out_frames*self.basis_kernels.shape[1]).to(device))
        self.bias.data.uniform_(-stdv, stdv)
        
    def get_weight_constraint(self):
        return torch.mean(torch.abs(self.relaxed_weights[...,:-1] - self.relaxed_weights[...,1:])) #torch.roll()

    def forward(self, x):
        conv_filters = torch.einsum('bpqk,obik->opiqk', self.basis_kernels.to(self.relaxed_weights.device), self.relaxed_weights) 
        conv_filters = conv_filters.reshape(conv_filters.shape[0]*conv_filters.shape[1],
                                            conv_filters.shape[2]*conv_filters.shape[3], 
                                            self.kernel_size, self.kernel_size)
        
        return F.conv2d(x, conv_filters, self.bias, padding = 1)
        
class Relaxed_TR_SteerConv(nn.Module):
    def __init__(self, in_frames, out_frames, kernel_size, N, num_banks, h_size, w_size, first_layer = False, last_layer = False):
        super(Relaxed_TR_SteerConv, self).__init__()
        self.convs = nn.Sequential(*[Relaxed_R_SteerConv(in_frames = in_frames, out_frames = out_frames, 
                                                       kernel_size = kernel_size, N = N, first_layer = first_layer, 
                                                       last_layer = last_layer).to(device) for i in range(num_banks)])
        
        self.combination_weights = nn.Parameter(torch.ones(h_size, w_size, num_banks).float().to(device)/num_banks)
        
        #self.activation = nn.ReLU()
        self.kernel_size = kernel_size
        self.pad_size = (kernel_size-1)//2
        self.h_size = h_size
        self.w_size = w_size
        self.last_layer = last_layer
        self.num_banks = num_banks
            

    def get_weight_constraint(self):
        return sum([layer.get_weight_constraint() for layer in self.convs])
        
    def forward(self, x):
        outs = torch.stack([self.convs[i](x) for i in range(self.num_banks)], dim  = 0)
        
        # Compute Convolution
        out = torch.einsum("ijr, rboij -> boij", self.combination_weights, outs)
        
        
        if self.last_layer:
            return out
        else:
            return self.convs[0].relu(escnn.nn.GeometricTensor(out, self.convs[0].feat_type_hid)).tensor
        
        
class Relaxed_TR_SteerConvNet(torch.nn.Module):
    def __init__(self, in_frames, out_frames, hidden_dim, kernel_size, num_layers, N, num_banks, h_size, w_size, alpha = 1):
        super(Relaxed_TR_SteerConvNet, self).__init__()
        self.alpha = alpha

        layers = [Relaxed_TR_SteerConv(in_frames = in_frames, out_frames = hidden_dim, 
                                       kernel_size = kernel_size, N = N, num_banks = num_banks,
                                       h_size = h_size, w_size = w_size, first_layer = True, last_layer = False)]
        
        layers += [Relaxed_TR_SteerConv(in_frames = hidden_dim, out_frames = hidden_dim, 
                                        kernel_size = kernel_size, N = N, num_banks = num_banks,
                                        h_size = h_size, w_size = w_size, first_layer = False, last_layer = False) 
                   for i in range(num_layers-2)]
        
        layers += [Relaxed_TR_SteerConv(in_frames = hidden_dim, out_frames = out_frames, 
                                        kernel_size = kernel_size, N = N, num_banks = num_banks,
                                        h_size = h_size, w_size = w_size, first_layer = False, last_layer = True) ]
        self.model = torch.nn.Sequential(*layers)
        
    def rot_vector(self, inp, theta):
        #inp shape: c x 2 x 64 x 64
        theta = torch.tensor(theta).float().to(device)
        rot_matrix = torch.tensor([[torch.cos(theta), -torch.sin(theta)], [torch.sin(theta), torch.cos(theta)]]).float().to(device)
        out = torch.einsum("ab, bc... -> ac...",(rot_matrix, inp.transpose(0,1))).transpose(0,1)
        return out
    
    def get_rot_mat(self, theta):
        theta = torch.tensor(theta).float().to(device)
        return torch.tensor([[torch.cos(theta), -torch.sin(theta), 0],
                             [torch.sin(theta), torch.cos(theta), 0]]).float().to(device)

    def rot_img(self, x, theta):
        rot_mat = self.get_rot_mat(theta)[None, ...].float().repeat(x.shape[0],1,1)
        grid = F.affine_grid(rot_mat, x.size()).float()
        x = F.grid_sample(x, grid)
        return x.float()
   
    def get_weight_constraint(self):
        return self.alpha * sum([layer.get_weight_constraint() for layer in self.model])
    
        
    def forward(self, xx):
        return self.model(xx)
    

### Figure 4 test file

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import random
from torch.utils import data
#from models import E2CNN, Relaxed_Rot_SteerConvNet, ConvNet
#from utils import Dataset, train_epoch, eval_epoch, get_lr
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


print("Data Equiv Errors: | {}".format([0.0, 0.17, 0.313, 0.435, 0.541, 0.63, 0.709, 0.78, 0.84, 0.896]))

hidden_dim = 64
num_layers = 5
out_length = 6
alpha = 0 
input_length = 1
batch_size = 32
num_epoch = 1000
learning_rate = 0.001
decay_rate = 0.9
mid = input_length + 2
seed = 0

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Split time ranges
train_time = list(range(0, 30))
valid_time = list(range(30, 40))

def rot_vector(inp, theta):
    #inp shape: c x 2 x 64 x 64
    theta = torch.tensor(theta).float().to(device)
    rot_matrix = torch.tensor([[torch.cos(theta), -torch.sin(theta)], [torch.sin(theta), torch.cos(theta)]]).float().to(device)
    out = torch.einsum("ab, bc... -> ac...",(rot_matrix, inp.transpose(0,1))).transpose(0,1)
    return out

def get_rot_mat(theta):
    theta = torch.tensor(theta).float().to(device)
    return torch.tensor([[torch.cos(theta), -torch.sin(theta), 0],
                         [torch.sin(theta), torch.cos(theta), 0]]).float().to(device)

def rot_img(x, theta):
    rot_mat = get_rot_mat(theta)[None, ...].float().repeat(x.shape[0],1,1)
    grid = F.affine_grid(rot_mat, x.size()).float()
    x = F.grid_sample(x, grid)
    return x.float()

def rot_field(x, theta):
    x_rot = torch.cat([rot_img(rot_vector(x, theta)[:,:1],  theta), 
                       rot_img(rot_vector(x, theta)[:,-1:], theta)], dim = 1)
    return x_rot


for model_name in ["E2CNN", "ConvNet", "RSteer"]:#
    equiv_error_lst = []
    for level in range(1):
        min_rmse = 1e8
        data_direc = "figure_4_data/E_" + str(level)
        train_task = [0, 1, 2, 3]

        train_set = Dataset(input_length = input_length, 
                            mid = mid, 
                            output_length = out_length,
                            direc = data_direc, 
                            task_list = train_task, 
                            sample_list = train_time, 
                            stack = True)

        valid_set = Dataset(input_length = input_length, 
                            mid = mid, 
                            output_length = out_length, 
                            direc = data_direc, 
                            task_list = train_task,
                            sample_list = valid_time, 
                            stack = True)

        train_loader = data.DataLoader(train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
        valid_loader = data.DataLoader(valid_set, batch_size = batch_size, shuffle = False, num_workers = 2) 
        test_loader = data.DataLoader(valid_set, batch_size = 1, shuffle = False, num_workers = 2)    


        if model_name == "E2CNN":
            model = nn.DataParallel(E2CNN(in_frames= input_length, 
                                          out_frames = 1, 
                                          hidden_dim = hidden_dim,
                                          kernel_size = 3, 
                                          num_layers = num_layers,
                                          N = 4).to(device))

        elif model_name == "RSteer":
            model = nn.DataParallel(Relaxed_Rot_SteerConvNet(in_frames = input_length, 
                                                             out_frames = 1, 
                                                             hidden_dim = hidden_dim//2, 
                                                             kernel_size = 3, 
                                                             num_layers = num_layers, 
                                                             N = 4, 
                                                             alpha = alpha).to(device))

        elif model_name == "ConvNet":
            model = nn.DataParallel(ConvNet(in_channels = input_length*2, 
                                            out_channels = 2,
                                            hidden_dim = hidden_dim,
                                            kernel_size = 3, 
                                            num_layers = num_layers).to(device))

        elif model_name == "RPP":
            model = nn.DataParallel(Rot_RPPNet(in_frames = input_length,
                                               out_frames = 1,
                                               hidden_dim = hidden_dim,
                                               kernel_size = 3, 
                                               num_layers = num_layers,
                                               N = 4).to(device))
        elif model_name == "Lift":
            model = nn.DataParallel(Lift_Rot_Expansion(in_frames = input_length,
                                                       out_frames = 1, 
                                                       kernel_size = 3, 
                                                       encoder_hidden_dim = hidden_dim//2, 
                                                       backbone_hidden_dim = hidden_dim//2, 
                                                       N = 4).to(device))



        optimizer = torch.optim.Adam(model.parameters(), learning_rate,betas=(0.9, 0.999), weight_decay=4e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=decay_rate)
        loss_fun = torch.nn.MSELoss()

        ########################################################################################################################################################

        train_rmse = []
        valid_rmse = []
        best_model = 0

        for i in range(num_epoch):
            start = time.time()

            model.train()
            train_rmse.append(train_epoch(train_loader, model, optimizer, loss_fun))

            model.eval()
            mse, preds, trues = eval_epoch(valid_loader, model, loss_fun)
            valid_rmse.append(mse)

            if valid_rmse[-1] < min_rmse:
                min_rmse = valid_rmse[-1] 
                best_model = model

            end = time.time()

            # Early Stopping
            if (len(train_rmse) > 50 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
                    break       

            scheduler.step()


        equiv_errors = []
        with torch.no_grad():
            for xx, yy in test_loader:
                xx = xx.to(device)
                orig_pred = best_model(xx).reshape(-1, 2, xx.shape[-2], xx.shape[-1])

                for angle in [np.pi/2, np.pi, np.pi/2*3]:
                    rho_inp = rot_field(xx.reshape(-1, 2, xx.shape[-2], xx.shape[-1]), angle).to(device)
                    rho_inp = rho_inp.reshape(1, -1, xx.shape[-2], xx.shape[-1])
                    rho_inp_outs = best_model(rho_inp).reshape(-1, 2, xx.shape[-2], xx.shape[-1])
                    equiv_errors.append(torch.mean(torch.abs(rho_inp_outs - rot_field(orig_pred, angle))).data.cpu())
        equiv_error_lst.append(np.round(np.mean(equiv_errors),3))

    print("{} Equiv Errors: | {}".format(model_name, equiv_error_lst))
    